# Tutorial

This tutorial assumes that you have gone through the first [Hotcent](https://gitlab.com/mvdb/hotcent) tutorial to generate an ```Si-Si_no_repulsion.skf``` file. In any case, this file is also included in the present folder.
<br><br>
We will now complete the parametrization exercise by fitting the remaining energy term (the so-called repulsive energy) to a DFT volume scan of bulk silicon.
<br><br>
Then we will briefly test the transferability of the resulting model by looking at several 2D forms of silicon. Lastly, it will be shown how to use it in a global optimization run for Si$_7$ clusters.

## Parametrization for Si

### Two-body repulsion

The following script creates a 'training.db' database which TANGO
uses to retrieve structures and their (DFT) energies. To save precious
tutorial time, these DFT energies have here been precalculated
(using GPAW with the LDA functional and a 400 eV plane wave basis set,
see ```run_gpaw_diamond_si.py```).

More information and examples on using such databases in ASE can be found
[here](https://wiki.fysik.dtu.dk/ase/ase/db/db.html) and
[here](https://wiki.fysik.dtu.dk/ase/tutorials/db/db.html).

In [ ]:
import os
import numpy as np
from ase.db import connect
from ase.build import bulk
from ase.data import atomic_numbers, covalent_radii
from ase.calculators.singlepoint import SinglePointCalculator

dbfile = 'training.db'
assert not os.path.exists(dbfile), 'Please remove the existing training.db file!'
db = connect(dbfile)

rcov = covalent_radii[atomic_numbers['Si']]
a0 = 2 * rcov * 4. / np.sqrt(3)

# Different fractions we will multiply the 'a0' lattice constant with:
fractions = np.arange(0.9, 1.31, 0.05)
# Associated total energies calculated with GPAW (LDA, 400 eV plane wave basis):
energies = [-8.156546300399613, -10.424474986302807, -11.546688457623372,
            -11.884715210546915, -11.648474045051344, -11.166297858643642,
            -10.518097847630376, -9.800655996150622, -9.06797134883969]
f = np.zeros((2, 3))

for x, e in zip(fractions, energies):
    atoms = bulk('Si', 'diamond', a=a0 * x)
    calc = SinglePointCalculator(atoms, energy=e, forces=f)
    atoms.set_calculator(calc)    
    db.write(atoms, relaxed=1, gaid=0)

print('Done!')

Before the actual fitting can begin, we furthermore need to
define how the required DFTB calculations (handled by TANGO)
are to be performed. For reasons of clarity and flexibility,
this is done by creating a custom Calculator class which is
only supposed to accept a limited number of (keyword) arguments
and where the remaining parameters are internally defined.
Have a look at the ```dftbplus_calc.py``` file which defines
this custom calculator.

Running the cell below will fit a pairwise repulsive potential
based on the total energy data in the ```training.db``` file.
Choosing the ```exp_poly``` mode means we will be using the
following ansatz for the potential:

$V_\mathrm{rep}(r) = \begin{cases} 
\exp \left( -a_1 \, r  \, + \,  a_2 \right) + a_3 &\mbox{     }0 \leq r < r_\mathrm{min} \\
\sum_{i=2}^p c_i\,(r_\mathrm{cut} - r)^i &\mbox{     }r_\mathrm{min} \leq r < r_\mathrm{cut}\\
0&\mbox{     }  r \geq r_\mathrm{cut}
\end{cases}$

The polynomial coefficients $c_i$ will be fitted by linear regression.
The parameters $a_1,a_2,a_3$ for the short-range exponential repulsion
are determined afterwards based on the zeroth, first, and second
derivatives of the polynomial curve evaluated at $r_\mathrm{min}$.
The repulsive potential is shown in `Si-Si.pdf` and is also appended
to the Slater-Koster tables in the `Si-Si.skf` file.

In [ ]:
%env DFTB_COMMAND='/home/maxime/Downloads/dftbplus-19.1.x86_64-linux/bin/dftb+'  # modify this

In [ ]:
from ase.data import atomic_numbers, covalent_radii
from dftbplus_calc import DftbPlusCalculator
from tango import TANGO

elements = ['Si']
mam = {'Si': 1}           # the maximum angular momenta for each element
mode = 'exp_poly'         # the functional form (exponential decay at short distances, then a polynomial)
fit_constant = 'element'  # allow for one constant energy shift for each element
kBT = 2.0                 # energy which defines the Boltzmann weights from the (relative) cohesive energies
kptdensity = 2.5          # the k-point density in points per Ang^-1
dbfiles = ['training.db']
force_scaling = 0.0       # how the forces should be weighted w.r.t. the forces;
                          # this does not matter here because the forces are zero by symmetry

rcov = covalent_radii[atomic_numbers['Si']]
rcuts = {'Si-Si': 1.5 * 2 * rcov}  # 'outer' cutoff as approximately 1.5 times the NN distance
rmins = {'Si-Si': 0.8 * 2 * rcov}  # 'inner' cutoff below which we switch to exponential repulsion
powers = {'Si-Si': range(2, 7)}    # standard (rcut-r)**2 + ... + (rcut-r)**6 polynomial

calc = TANGO(elements,
             DftbPlusCalc=DftbPlusCalculator,
             kptdensity=kptdensity,
             rmins=rmins,
             rcuts=rcuts,
             powers=powers,
             fit_constant=fit_constant,
             kBT=kBT,
             mode=mode,
             force_scaling=force_scaling,
             maximum_angular_momenta=mam,
             )

residual = calc.fit_repulsion(dbfiles, run_checks=True)
print('Residual:', residual)

### Transferability test: flat and buckled silicene

By construction, the model reproduces well the (isotropic) expansion and compression of bulk diamond Si. As a first transferability test, we will consider different forms of two-dimensional silicon. In analogy with graphene, one can consider a perfectly flat honeycomb-like structure. In LDA-DFT, however, a buckled configuration is energetically preferred by a small amount (34 meV). What does the DFTB model give?

In [ ]:
import os
import numpy as np
from ase.io import write
from ase.lattice.hexagonal import Graphene
from ase.optimize import BFGS
from ase.constraints import ExpCellFilter
from dftbplus_calc import DftbPlusCalculator

os.environ['DFTB_PREFIX'] = './'

rcov = covalent_radii[atomic_numbers['Si']]
lc = 2 * np.sqrt(3) * rcov
atoms = Graphene(symbol='Si', latticeconstant=[lc, 12.])
atoms.center()

calc = DftbPlusCalculator(atoms, kpts=(5, 5, 1), use_spline=True,
                          maximum_angular_momenta={'Si': 1})
atoms.set_calculator(calc)

ecf = ExpCellFilter(atoms, mask=(1, 1, 0, 0, 0, 1))
dyn = BFGS(ecf, logfile='-')
dyn.run(fmax=0.01)
write('relaxed_silicene_flat.traj', atoms)

atoms.positions[:,2] += np.array([0.5, -0.5])
atoms.set_calculator(calc)
dyn = BFGS(ecf, logfile='-')
dyn.run(fmax=0.01)
write('relaxed_silicene_buckled.traj', atoms)

### Tranferability test: 'zigzag-dumbbell' silicene

In contrast with two-dimensional carbon, however, the putative global minium structure of ultrathin silicon
is not graphene-like, as reported in [this study by Borlido and coworkers](https://dx.doi.org/10.1088/2053-1583/aab9ea).

The initial geometry for this so-called 'zigzag-dumbbell' structure (used in the script below) was again obtained after LDA-DFT relaxation, with a final energy lying 262 meV/atom below that of buckled silicene.

How does the relaxed DFTB geometry compare with the initial one (see ```opt_zigzag_dumbbell.traj```). Is it indeed more stable than the 'flat' and 'buckled' configurations?

In [ ]:
import os
import numpy as np
from ase import Atoms
from ase.io import write
from ase.optimize import BFGS
from ase.constraints import ExpCellFilter
from dftbplus_calc import DftbPlusCalculator


os.environ['DFTB_PREFIX'] = './'

# Coordinates taken from the following publication:
# Borlido et al., "The ground state of two-dimensional silicon",
#   2D Mater. 5, 035010 (2018), doi:10.1088/2053-1583/aab9ea
# and reoptimized with GPAW (LDA, 400 eV plane wave basis)
atoms = Atoms('Si10', 
              pbc=True,
              cell=(6.3245948276063171, 7.3351795905906787, 15., 90., 90., 90.),
              positions=[[2.5238234215085167,  1.8915376909714172,  7.5000000000042730],
                         [4.4402893598836890,  1.8913405945624528,  8.8460149135358996],
                         [1.2984521401199047,  3.8644391832367875,  7.5000000000030678],
                         [2.1934070307608962,  5.5591598065851038,  6.1539843200547191],
                         [4.4402893598687605,  1.8913405945582829,  6.1539850864607581],
                         [2.1934070307611422,  5.5591598065878962,  8.8460156799451646],
                         [5.3352644364352750,  0.1966184137746097,  7.4999999999998295],
                         [1.2984323446246357,  7.2538814320204086,  7.4999999999998987],
                         [5.3352446040859824,  3.5860618244812916,  7.4999999999982716],
                         [4.1098722380006052,  5.5589627502960530,  7.4999999999981020]])

calc = DftbPlusCalculator(atoms, kpts=(4, 3, 1), use_spline=True,
                          maximum_angular_momenta={'Si': 1})
atoms.set_calculator(calc)
atoms.set_calculator(calc)

# Local geometry optimization
ecf = ExpCellFilter(atoms, mask=(1, 1, 0, 0, 0, 1))
dyn = BFGS(ecf, logfile='-', trajectory='opt_zigzag_dumbbell.traj')
dyn.run(fmax=0.01)
write('relaxed_zigzag_dumbbell.traj', atoms)

#### Bonus tasks:
- Other possible test systems include the silicon dimer (gas phase Si$_2$) and the (un)reconstructed Si(100) surface.
- How does the [pbc-0-3 parameter set](http://www.dftb.org/parameters/download/pbc/pbc-0-3-cc/) perform on these tests?
- As an additional exercise, you may try to an SKF file which also takes into account the unoccupied Si 3d states, refit the repulsive potential, and examine its performance.

## Global optimization and DFTB

Solving global structure optimization problems may require
several thousands (or more) 'attempts', each requiring the local
minimization of a trial structure. This strongly limits the
system sizes that can be addressed with this method at the
DFT level. The speed-up provided by DFTB, however, substantially
lowers the computational cost and/or gives access to more relevant
global optimization (GO) problems involving a larger number of atoms.

The spirit of the TANGO approach is to start by creating an initial,
possibly rather crude, parametrization (based on DFT calculations on
either randomly generated or manually chosen structures), which can
then be refined after partial exploration of the potential energy
landscape using the initial parameter set.

The simple parametrization created based on diamond Si, for example,
should be appropriate as initial guess for the global optimization of
silicon clusters. The script below provides a (somewhat) minimalistic example
using a genetic algorithm (GA) to search for the minimum-energy Si$_7$
clusters. More advanced examples which also include DFTB reparametrization,
can be found among the [examples on gitlab](https://gitlab.com/mvdb/tango/tree/master/examples).

Normally, without too much bad luck, you should retrieve a starfish-shaped
cluster as the global minimum structure (the first structure in
```run000/current_population.traj```). The same arrangement has also
been proposed on the basis of DFT calculations (see e.g. [here]()).
The whole calculation should take about half a minute.

Note that, for more complicated 'real-life' GO problems, many more iterations
are needed (hundreds, thousands, ...). In addition, multiple independent GA
runs should be run in parallel so as to maximize the probability of finding
the global minimum (and the next-lowest local minima, which are usually
also of interest).

Further information and examples regarding the GA framework in ASE
can be found on [this page](https://wiki.fysik.dtu.dk/ase/ase/ga/ga.html).

In [ ]:
import os
import sys
from random import random, seed
import numpy as np
from time import time
from ase import Atoms
from ase.io import read, write
from ase.data import atomic_numbers
from ase.ga import get_raw_score
from ase.ga.data import DataConnection, PrepareDB
from ase.ga.population import Population
from ase.ga.utilities import (get_all_atom_types, closest_distances_generator,
                              atoms_too_close)
from ase.ga.ofp_comparator import OFPComparator
from ase.ga.startgenerator import StartGenerator
from ase.ga.cutandsplicepairing import CutAndSplicePairing
from ase.ga.standardmutations import RattleMutation
from ase.ga.offspring_creator import OperationSelector
from tango.relax_utils import relax_standard, finalize
from dftbplus_calc import DftbPlusCalculator


def relax_one(atoms):
    """
    This method defines how to locally minimize a given atoms object.
    """
    print('Starting relaxation')
    clock = time()

    # Define the DFTB+ calculator:
    calc = DftbPlusCalculator(atoms, kpts=(1, 1, 1), use_spline=True,
                              maximum_angular_momenta={'Si': 1})

    # Start the actual relaxation using the
    # tango.relax_utils.relax_precon method.
    # This wraps around the ASE optimizers and also
    # takes care of setting the raw score etc.:
    try:
        atoms = relax_standard(atoms, calc, fmax=2e-2, variable_cell=False,
                               logfile=None, trajfile=None, optimizer='BFGS')
    except (IOError, UnboundLocalError):
        # the DFTB+ ASE calculator throws an IOError or
        # UnboundLocalError in case it couldn't find the
        # 'results.tag' output file, which may sporadically
        # happen due to SCC converge issues when e.g. a Si7
        # cluster is breaking into fragments. We simply
        # handle this by aborting the relaxation and
        # assigning a very high energy to the structure:
        finalize(atoms, energy=1e9, forces=None, stress=None)

    print('Relaxing took %.3f seconds.' % (time() - clock), flush=True)
    return atoms


def run_ga(n_to_test):
    """
    This method specifies how to run the GA once the
    initial random structures have been stored in godb.db.
    """
    # Various initializations:
    population_size = 10  # maximal size of the population
    da = DataConnection('godb.db')
    atom_numbers_to_optimize = da.get_atom_numbers_to_optimize()  # = [14] * 7
    n_to_optimize = len(atom_numbers_to_optimize)  # = 7
    # This defines how close the Si atoms are allowed to get
    # in candidate structures generated by the genetic operators:
    blmin = closest_distances_generator(atom_numbers_to_optimize,
                                        ratio_of_covalent_radii=0.4)
    # This is our OFPComparator instance which will be
    # used to judge whether or not two structures are identical:
    comparator = OFPComparator(n_top=None, dE=1.0, cos_dist_max=1e-3,
                               rcut=10., binwidth=0.05, pbc=[False]*3,
                               sigma=0.1, nsigma=4, recalculate=False)

    # Defining a typical combination of genetic operators:
    pairing = CutAndSplicePairing(da.get_slab(), n_to_optimize, blmin)
    rattlemut = RattleMutation(blmin, n_to_optimize, rattle_prop=0.8,
                               rattle_strength=1.5)
    operators = OperationSelector([2., 1.], [pairing, rattlemut])

    # Relax the randomly generated initial candidates:
    while da.get_number_of_unrelaxed_candidates() > 0:
        a = da.get_an_unrelaxed_candidate()
        a = relax_one(a)
        da.add_relaxed_step(a)

    # Create the population
    population = Population(data_connection=da,
                            population_size=population_size,
                            comparator=comparator,
                            logfile='log.txt')
    current_pop = population.get_current_population()

    # Test n_to_test new candidates
    for step in range(n_to_test):
        print('Starting configuration number %d' % step, flush=True)

        a3 = None
        while a3 is None:
            a1, a2 = population.get_two_candidates()
            a3, description = operators.get_new_individual([a1, a2])

        da.add_unrelaxed_candidate(a3, description=description)
        a3 = relax_one(a3)
        da.add_relaxed_step(a3)

        population.update()
        best = population.get_current_population()[0]
        print('Highest raw score at this point: %.3f' % get_raw_score(best))

    print('GA finished after step %d' % step)
    write('all_candidates.traj', da.get_all_relaxed_candidates())
    write('current_population.traj', population.get_current_population())


def prepare_ga(dbfile='godb.db', N=20):
    """
    This method creates a database with the desired number
    of randomly generated structures.
    """ 
    Z = atomic_numbers['Si']
    atom_numbers = [Z] * 7

    # This dictionary will be used to check that the shortest
    # Si-Si distances are above a certain threshold 
    # (here 1.5 Angstrom):
    blmin = {(Z, Z): 1.5}  

    # This defines the cubic simulation cell. In case there
    big_cell = np.identity(3) * 16.
    slab = Atoms('', cell=big_cell)

    # This defines the smaller box in which the initial
    # coordinates are allowed to be generated
    density = 0.1  # in atoms per cubic Angstrom
    L = np.cbrt(len(atom_numbers) / density)
    small_cell = np.identity(3) * L
    p0 = 0.5 * np.diag(big_cell - small_cell)
    box = [p0, small_cell]

    # Seed the random number generators using the system time,
    # to ensure that no two runs produce the same results:
    np.random.seed()
    seed()

    # Generate the random structures and add them to the database:
    sg = StartGenerator(slab=slab, atom_numbers=atom_numbers,
                        closest_allowed_distances=blmin,
                        box_to_place_in=box)

    da = PrepareDB(db_file_name=dbfile,
                   simulation_cell=slab,
                   stoichiometry=atom_numbers)

    for i in range(N):
        a = sg.get_new_candidate()
        da.add_unrelaxed_candidate(a)

    return


if __name__ == '__main__':
    cwd = os.getcwd()
    os.environ['DFTB_PREFIX'] = cwd

    rundir = 'run000'
    if not os.path.exists(rundir):
        os.mkdir(rundir)
    os.chdir(rundir)
 
    dbfile = 'godb.db'
    if not os.path.exists(dbfile):
        prepare_ga(dbfile=dbfile, N=5)

    run_ga(10)

    os.chdir(cwd)